## Functions that have proved useful

In [ ]:
from functools import reduce
def factors(n):
    return set(reduce(list.__add__,([i, n//i] for i in range(1, int(n**0.5) + 1) if n % i == 0)))

def rebin(ys, xs, rebinfactor):
    outys = ys.reshape(-1, rebinfactor).mean(axis=1)
    outxs = xs.reshape(-1, rebinfactor).mean(axis=1)
    return outys, outxs

rebinfactors = np.sort(np.array(list(factors(len(y2)))))
print(f'possible factors are ', rebinfactors)

rebinf = rebinfactors[6]
print(f'using factor {rebinf}')

In [ ]:
# fitting functions
from scipy.optimize import curve_fit

def func_log(x, a, b):
    """Return values from a general log function."""
    return a * np.log(b * x)

def func_exp(x, a, t, c): ## a is vertical "compression" factor (-) value reflects across x axis, tau is t, c is vertical shift
    return a * np.exp(x/-t) + c

def func_gauss(x,amp,mu,sigma): # Gaussian function model
    return amp*np.exp(-(x-mu)**2/2*sigma**2)

In [ ]:
## general function for reduced chi square for a given fit function. sigma is important to estimate.

def chi_square_dof(fit_func, xdata, ydata, popt, sigma=None):
    """
    Calculate chi-square and degrees of freedom for a fit.

    Parameters:
        fit_func : callable
            The model function, f(x, ...).
        xdata : array_like
            The independent variable where the data is measured.
        ydata : array_like
            The dependent data — measured values.
        popt : array_like
            Optimal values for the parameters from curve_fit.
        sigma : array_like or None
            Standard deviation errors of ydata. If None, equal weights assumed.

    Returns:
        chi2 : float
            Chi-square statistic.
        dof : int
            Degrees of freedom.
        reduced_chi2 : float
            Reduced chi-square (chi2 / dof)
    """
    yfit = fit_func(xdata, *popt)
    residuals = ydata - yfit
    if sigma is None:
        chi2 = np.sum(residuals**2)
    else:
        chi2 = np.sum((residuals / sigma)**2)
    dof = len(ydata) - len(popt)
    reduced_chi2 = chi2 / dof
    return chi2, dof, reduced_chi2

In [ ]:
## plotting param

plt.rcParams["font.family"] = "sans-serif"
plt.xlabel(r'Temperature [$\degree$C]', size='x-large')
plt.ylabel('Voltage [V]', size='x-large')
plt.legend()
plt.title(r'Voltage Change with Coil Temperature', size='x-large')

plt.tick_params(axis='x', labelsize = 'large')
plt.tick_params(axis='y', labelsize = 'large')
plt.ticklabel_format(axis='both', style='plain', scilimits=(0,0))
plt.grid(True, which='both',axis = 'both', alpha = 0.75, ls = ':')
plt.minorticks_on()

plt.show()

In [ ]:
## PLotting histograms example, with fitting

%matplotlib inline
r = [-0.08,0.08]
b = 100

all_NaI_amp = all_amp_norm.flatten()
# all_NaI_amp = np.sum(all_amp_norm, axis = 0)
# print(len(testsum))
plt.title(rf'All 24 NaI detectors, different methods', fontsize = 'xx-large')
mean = np.round(np.mean(all_NaI_amp),6)
std = np.round(np.std(all_NaI_amp)/(np.sqrt(numruns_tot_D*24)),6)

n_amp, bins_amp, patches_amp = plt.hist(all_NaI_amp,bins=b,range=r, label = f'Amplitude method\nmean = {mean}, (std/ $\sqrt{{N_R*N_D}}$) = {std}')
print(mean)
print(np.mean(sum_ch_amp))

all_NaI_integral = all_integral_norm.flatten()
# all_NaI_integral = np.sum(all_integral_norm, axis = 0)
# print(len(testsum))
# plt.title(rf'maybe all 24 detectors using integral over FWHM', fontsize = 'xx-large')
mean2 = np.round(np.mean(all_NaI_integral),6)
std2 = np.round(np.std(all_NaI_integral)/(np.sqrt(numruns_tot_D*24)),6)

n_int, bins_int, patches_int = plt.hist(all_NaI_integral,bins=b,range=r, label = f'Integral method\nmean = {mean2}, std = {std2}', alpha =0.9, hatch='--',histtype='step')

def gauss(x,amp,mu,sigma): # deinition of function
    return amp*np.exp(-(x-mu)**2/2*sigma**2)

y_amp = n_amp
y_int = n_int
x = np.linspace(r[0],r[1], b)

popt_amp,pcov_amp=curve_fit(gauss,x,y_amp,p0=[100,0,2]) # popt= optimize parameter
plt.plot(x,gauss(x,*popt_amp),color='blue',lw=2.0,ls = '--',label=f'Amplitude Gaussian fit mean={np.round(popt_amp[0],5)}, mu={np.round(popt_amp[1],5)}')
popt_int,pcov_int=curve_fit(gauss,x,y_int,p0=[100,0,2]) # popt= optimize parameter
plt.plot(x,gauss(x,*popt_int),color='orange',lw=2.0,ls = '--',label=f'Integral Gaussian fit mean={np.round(popt_int[0],5)}, mu={np.round(popt_int[1],5)}')
# print('amp',popt[0]) # amp
# print('meam',popt[1])# mean mu
# print('sigma',popt[2]) # sigma


plt.xlabel('asymmetry absolute')
plt.legend()
plt.show()
print(mean2)
# print(np.mean(sum_ch_integral))


### more niche

In [ ]:
## convert HH:MM:SS.0 to seconds
from datetime import datetime

def time_to_seconds(time_str):
    time_obj = datetime.strptime(time_str, '%H:%M:%S.%f')
    seconds = (time_obj - datetime(1900, 1, 1)).total_seconds()
    return int(seconds)

t_sec = []
for i in range(1,len(df_t['T-T0 [HH:MM:ss.0]'])): ## 1 skips the power supply off row
    sec = time_to_seconds(df_t['T-T0 [HH:MM:ss.0]'][i])
#     print(df_t['T-T0 [HH:MM:ss.0]'][i], sec)
    t_sec.append(sec)

In [ ]:
## fields of coils
# Physical constant
mu_0 = 4 * np.pi * 1e-7  # Vacuum permeability (T·m/A)

def single_loop_field(z, z0, R, I, N):
    """
    Magnetic field along the z-axis from a single circular current loop.

    Parameters:
    - z: numpy array of positions along the z-axis
    - z0: z-position of the center of the loop
    - R: radius of the loop (meters)
    - I: current in the loop (Amperes)
    - N: number of turns in the loop

    Returns:
    - Bz: magnetic field at each z position (Tesla)
    """
    return (mu_0 * N * I * R**2) / (2 * ((R**2 + (z - z0)**2)**(1.5)))

def total_magnetic_field(z, coil_configs):
    """
    Compute total magnetic field along the z-axis from multiple coils.

    Parameters:
    - z: numpy array of z positions
    - coil_configs: list of dictionaries, each with keys:
        {'z0': float, 'R': float, 'I': float, 'N': int}

    Returns:
    - Bz_total: total magnetic field along z-axis (Gauss)
    """
    Bz_total = np.zeros_like(z)
    for coil in coil_configs:
        Bz_total += single_loop_field(
            z=z,
            z0=coil['z0'],
            R=coil['R'],
            I=coil['I'],
            N=coil['N']
        )
    return Bz_total*10000

In [ ]:
## get memory sizes of arrays 
memory_usage = sys.getsizeof(all_D)
print(f"Memory usage of the list: {memory_usage} bytes")

def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))